<a href="https://colab.research.google.com/github/mwinzie/Football-Analysis-Mchezopesa-Limited/blob/master/Moringa_Data_Science_Core_W6_Independent_Project_Agnes_Mwinzi_DataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#loading the necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import datetime as dt

In [0]:
#loading the datasets
ranking = pd.read_csv('fifa_ranking.csv')
results = pd.read_csv('results.csv')

In [6]:
#previewing the first two observations in the ranking dataset
ranking.head(2)

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08


In [8]:
#previewing the first two observations in the results dataset
results.head(2)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False


In [9]:
#checking the informations contained in the datasets 
print(results.info())
print('************************')
print(ranking.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40839 entries, 0 to 40838
Data columns (total 9 columns):
date          40839 non-null object
home_team     40839 non-null object
away_team     40839 non-null object
home_score    40839 non-null int64
away_score    40839 non-null int64
tournament    40839 non-null object
city          40839 non-null object
country       40839 non-null object
neutral       40839 non-null bool
dtypes: bool(1), int64(2), object(6)
memory usage: 2.5+ MB
None
************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57793 entries, 0 to 57792
Data columns (total 16 columns):
rank                       57793 non-null int64
country_full               57793 non-null object
country_abrv               57793 non-null object
total_points               57793 non-null float64
previous_points            57793 non-null int64
rank_change                57793 non-null int64
cur_year_avg               57793 non-null float64
cur_year_avg_weighted      57793

Here we get to see the datatypes of the columns in the datasets. The results dataset has 40,839 rows and 9 columns while the ranking dataset has 57,793 rows and 16 columns. 

From the data observations in the Fifa ranking csv(ranking dataset), ranking began in the year 1993, so the data before 1993 may not be useful in my model and prediction, hence I shall drop all observations below 1993 in the results dataset.

In [0]:
#dropping rows in the results dataset that are below the year 1993.
#I shall check for uniformity in the date column and then drop
results['date'].unique().tolist()
#converting the date column to date time datatype
results.date = pd.to_datetime(results.date)

In [21]:
#checking to confirm the datatype change
results.dtypes

date          datetime64[ns]
home_team             object
away_team             object
home_score             int64
away_score             int64
tournament            object
city                  object
country               object
neutral                 bool
dtype: object

In [22]:
results['date'].head(2)

0   1872-11-30
1   1873-03-08
Name: date, dtype: datetime64[ns]

In [28]:
results.head(2)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,month
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1872,11
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,1873,3


In [0]:
#deleting the rows that are below 1993, when FIFA started ranking the teams
#results['year'] = results(['year']< 1993).dropna(how = 'all')
results = results.drop(results[(results.date < '1993-08-08')] .index)


In [43]:
ranking.tail(2)

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
57791,206,Somalia,SOM,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57792,206,Tonga,TGA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OFC,2018-06-07


In [42]:
results.tail(2)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,month
40837,2019-07-19,Tajikistan,North Korea,0,1,Intercontinental Cup,Ahmedabad,India,True,2019,7
40838,2019-07-20,Papua New Guinea,Fiji,1,1,Pacific Games,Apia,Samoa,True,2019,7


In [40]:
results.shape

(22924, 11)

In [41]:
ranking.shape

(57793, 16)

## Data Cleaning of the ranking dataset

In [10]:
#checking the columns in the ranking dataset
ranking.columns

Index(['rank', 'country_full', 'country_abrv', 'total_points',
       'previous_points', 'rank_change', 'cur_year_avg',
       'cur_year_avg_weighted', 'last_year_avg', 'last_year_avg_weighted',
       'two_year_ago_avg', 'two_year_ago_weighted', 'three_year_ago_avg',
       'three_year_ago_weighted', 'confederation', 'rank_date'],
      dtype='object')

In [13]:
#checking if there exists any null values
ranking.isnull().any()
#the dataset does not have any null values

rank                       False
country_full               False
country_abrv               False
total_points               False
previous_points            False
rank_change                False
cur_year_avg               False
cur_year_avg_weighted      False
last_year_avg              False
last_year_avg_weighted     False
two_year_ago_avg           False
two_year_ago_weighted      False
three_year_ago_avg         False
three_year_ago_weighted    False
confederation              False
rank_date                  False
dtype: bool

## Data Cleaning of the Results dataset

In [11]:
#checking the column names in the results dataset
results.columns

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral'],
      dtype='object')

In [14]:
#checking if there exists any null values
results.isnull().any()
#the dataset does not have any null values

date          False
home_team     False
away_team     False
home_score    False
away_score    False
tournament    False
city          False
country       False
neutral       False
dtype: bool

In [47]:
results.columns

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral', 'year', 'month'],
      dtype='object')

In [46]:
#creating a new column to show the winning team
winner = []
for i in range(len(results['home_team'])):
  if results['home_score'][i] > results['away_score'][i]:
    winner.append(results['home_team'][i])
  elif results['home_score'][i] < results['away_score'][i]:
    winner.append(results['away_team'][i])
  else:
    winner.append('draw')
results['winning_team'] = winner

KeyError: ignored

In [21]:
results.head(2)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winning_team
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,draw
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,England


In [0]:
#creating a new column showing the wins, loses and draws
